In [1]:

import numpy as np
import matplotlib.pylab as plt
import time
from scipy import optimize
from scipy import integrate
# from numba import jit
import pandas as pd

In [2]:
Dim = 3
x = np.random.normal(0,1,(1000,Dim))
x[:,1] += 2.
x[:,2] = 2*x[:,2]

print(x)

[[ 2.28083573  2.26162145  0.35658716]
 [ 0.12094913  0.87861188  0.29128127]
 [-0.66203521  4.31860248  0.67432902]
 ...
 [-0.62197979  0.30802022 -1.09960879]
 [ 0.05243607  2.5922898   1.08383146]
 [-1.32799317  2.32778813 -3.05154878]]


In [3]:
Library = np.ones((x.shape[0],1))
print(Library.shape)


for i in range(Dim): # linear
    Library = np.concatenate((Library,x[:,i].reshape(-1,1)),
                             axis = 1)
    
for i in range(Dim): # quadratic
    for j in range(i,Dim):
        Library = np.concatenate((Library,(x[:,i]*x[:,j]).reshape(-1,1)),
                             axis = 1)    

(1000, 1)


In [4]:
def D1(x):
    return(x[:,0] + 2*x[:,0]**2. + 0.01 * x[:,1]- 0.5*x[:,2]**2.)

D1(x[0,:].reshape(1,-1))

array([12.64429796])

In [5]:
D_on_x = D1(x)
print(D_on_x.shape, type(D_on_x))
print(D_on_x)

(1000,) <class 'numpy.ndarray'>
[ 1.26442980e+01  1.16570239e-01  3.03722342e-02 -2.63623692e+00
  1.33484523e+00 -6.50765244e-01  1.21533830e+00  6.28005268e-02
 -2.08001229e+00  9.27122753e-01  1.20831197e-01 -5.52883553e+00
 -1.52981790e+00 -1.03315031e+00  1.88338666e+00 -1.24584443e+00
 -3.48105117e+00 -9.81279818e-01 -4.93205642e-01 -1.32576455e-01
  5.43043827e-01  9.25853576e-01 -5.99321064e+00 -7.85354887e-02
 -1.13992542e+00  2.43747657e+00 -1.62220316e-01  8.93201128e-01
  2.41215896e+00 -3.62132726e-01 -2.42746832e-01 -1.91584650e+00
  2.71732253e-01 -8.24082139e+00 -2.91765497e+00 -3.82289843e-01
 -3.04893448e+00  7.30242814e-01 -1.13759538e+00  4.76561809e+00
  3.49897385e+00 -1.55252456e+00 -3.23571328e+00 -1.14204109e+00
 -1.02576314e+00 -4.36821736e+00  7.99799054e+00  2.17276990e-01
 -1.11142768e+00  2.21877135e+00 -4.59342859e-01 -9.84527179e-01
  1.13919178e+01 -1.49705925e-01 -2.01458844e+00  1.86753370e+00
  4.49749838e+00  6.79258244e+00  8.18628007e-01 -1.127728

### Ranges


In [6]:
x[0,:]

array([2.28083573, 2.26162145, 0.35658716])

In [7]:
Ranges = np.empty((Dim,2)) # minimum and Maximum
for i in range(Dim):
    Ranges[i,:] = [min(x[:,i]), max(x[:,i])]
print(Ranges)

[[-3.45273088  3.055354  ]
 [-1.5926956   5.14112613]
 [-6.23410324  7.16331157]]


### Simple Integral Approach

In [8]:
def Marginalise_over_One(x,i,f, Range_i, nx=100):
    # input: x0, x1, ....; i is the xi for which f is integrated over
    #        nx: number of increments for integration
    # note that the value for x_i is irrelevant for the algorithm
    # output: marginalised value for function f with respect to xi    
    Support = np.linspace(Range_i[0], Range_i[1], nx)
    dxi = Support[1] - Support[0]
    
    dimx = len(x)
    IntegrationArea = np.empty((nx,dimx))
    for j in range(dimx):
        if j != i: # for all xj input: always constant values
            IntegrationArea[:,j] = np.ones(nx)*x[j]
        elif j == i:
            IntegrationArea[:,j] = Support
            
    f_val = f(IntegrationArea)
    return(f_val.sum()*dxi /(Range_i[1]- Range_i[0]) )


In [9]:
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:] ,nx = 1000))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1,Range_i = Ranges[1,:], nx = 100000))

2.543173891572976
2.520262415072284
2.5177673303265213


In [10]:
def Integrand(x,i,f,Range_i, nx=100):
    return( (f(x.reshape(1,-1)) - Marginalise_over_One( x,i, f, Range_i, nx=100  ))**2. )

In [11]:
print(Integrand(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:], nx=100))

[17.2113558]


In [12]:

def One(x0,x1,x2):
    return(1.)

def integrand(x0,x1,x2,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2]), i, f , Range_i , nx))

In [13]:
Volume, verror = integrate.nquad(One,[[-3,3], [-1,5], [-7,6]])
print(Volume, verror)
for i in range(Dim):
    
    result, error = integrate.nquad(integrand, [[-3,3], [-1,5], [-7,6]],
                               args = (i, D1, Ranges[i,:], 100)) 
    print(i, result/Volume, error)

468.0 5.195843755245733e-12
0 32.97387161160642 1.7132708489943974e-10
1 0.008143333510543882 4.231148856764274e-14
2 43.53844212228774 2.2621894261421655e-10


In [14]:
Integrands = list([Integrand, integrand])
print(Integrands)

[<function Integrand at 0x7f664319a268>, <function integrand at 0x7f664319a9d8>]


## Now with the 6D-Coupled-Lorenz

In [15]:

def integrand6(x0,x1,x2,x3,x4,x5,x6,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2,x3,x4,x5,x6]), i, f , Range_i , nx))

In [16]:
BIC= pd.read_csv("Lorenz_Score_BIC.csv", index_col = 0)
print(BIC)
print(BIC["BIC"]==min(BIC["BIC"]))
logic = BIC["BIC"]==min(BIC["BIC"])
print(np.arange(3)[logic])

           BIC  Threshold
0  1876.523805       0.05
1  1747.523895       0.55
2  7867.645786       1.05
0    False
1     True
2    False
Name: BIC, dtype: bool
[1]


In [17]:

filename = 'CoupledLorenz_dt0.01' 
x = np.load(filename+'.npy')
print(x.shape)

(5000, 6)


In [18]:
BestAlphas = np.loadtxt('CoupledLorenz_Alpha_Done.txt')[BIC["BIC"]==min(BIC["BIC"]),:]

In [19]:
RangeL = np.empty((6,2))
Dim = x.shape[1]    # NOW NEW DIMENSION!
for i in range(Dim):
    RangeL[i,:] = [min(x[:,i]), max(x[:,i])] 
print(RangeL)

[[-19.32628135  21.26101002]
 [-24.5107052   27.94186795]
 [  1.          52.2962196 ]
 [-22.45140621  20.64704473]
 [-28.70851393  24.51948398]
 [  3.74703334  53.52157254]]


In [20]:
# let's only regard up to second order    
def poly(x,sigma):
    x_vec=np.array([1,x[0],x[1],x[2],x[3],x[4],x[5],  #7
                   x[0]**2., x[1]**2., x[2]**2., x[3]**2., x[4]**2., x[5]**2., # 6 terms
                    x[0]*x[1], x[0]*x[2], x[0]*x[3], x[0]*x[4], x[0]*x[5], # 5 terms
                    x[1]*x[2], x[1]*x[3], x[1]*x[4], x[1]*x[5], # 4 terms
                    x[2]*x[3], x[2]*x[4], x[2]*x[5], # 3 terms
                    x[3]*x[4], x[3]*x[5], x[4]*x[5]]) # 3 terms
    
    return np.dot(sigma,x_vec) # Total: 28 terms

#@jit
def D1(x):
    sigma = BestAlphas.flatten()[1:] # without noise parameters
    sigma=sigma.reshape((Dim,-1))
    function=np.zeros((len(x),Dim))
    for i in range(Dim):
        function[:,i]=poly(x.T,sigma[i])
    return function

In [21]:
poly(x.T,BestAlphas.flatten()[1:29])

/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


array([  0.91615775,  -2.33633244,   4.81708547, ...,  -6.31656045,
        -8.84006699, -11.14266855])

In [22]:
print(BestAlphas.shape)
print(BestAlphas.flatten().shape)


(1, 169)
(169,)


In [23]:
Sigma_Vec = BestAlphas.flatten()

def D1_j(x0,x1,x2,x3,x4,x5,j):
    # returns the j'th component of D1
    x_arr = np.array([x0,x1,x2,x3,x4,x5])
    return(D1(x_arr)[0,j])

for j in range(6):
    print(D1_j(x[0,0], x[0,1],x[0,2], x[0,3],x[0,4],x[0,5],j))
    
    
def D1_0(x):
    return(D1(x)[0,0])

D1_0(x[0,:])

0.9161577529163942
26.622084909093488
0.1684962347598935
15.586333694423445
161.0531733434817
76.04350914923002


0.9161577529163942

In [24]:
for j in range(6):
    print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = j, f = D1_0, Range_i = RangeL[j,:], nx=100))

1.9673706561506643
-2.525351804129916
0.009254118716327215
0.009254118716327184
0.009254118716327196
0.009254118716327213


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [25]:
def Marginalise_over_One(x,i,f, Range_i, nx=100):
    # input: x0, x1, ....; i is the xi for which f is integrated over
    #        nx: number of increments for integration
    # note that the value for x_i is irrelevant for the algorithm
    # output: marginalised value for function f with respect to xi    
    Support = np.linspace(Range_i[0], Range_i[1], nx)
    dxi = Support[1] - Support[0]
    
    dimx = len(x)
    IntegrationArea = np.empty((nx,dimx))
    for j in range(dimx):
        if j != i: # for all xj input: always constant values
            IntegrationArea[:,j] = np.ones(nx)*x[j]
        elif j == i:
            IntegrationArea[:,j] = Support
            
    f_val = f(IntegrationArea)
    return(f_val.sum()*dxi /(Ranges[i,1]- Ranges[i,0]) )

print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 1, f = D1_0, Range_i = RangeL[1,:], nx=100))

-19.6710286504666


/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [26]:
####

In [27]:
def One6(x0,x1,x2,x3,x4,x5):
    return(1)

In [ ]:
Volume, verror = integrate.nquad(One6,RangeL)
print(Volume, verror)


In [ ]:
print(RangeL)
V = 1
for i in range(6):
    V = V*(RangeL[i,1]-RangeL[i,0])
print(V)

In [ ]:
Marginalise_over_One

In [ ]:
print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))

In [ ]:
for i in range(Dim):
    
    print(i, Marginalise_over_One(x, i=0, D1_j()))

In [ ]:
def One2d(x0,x1):
    return([1,1])
print(One2d(1,1))
print(integrate.nquad(One2d,[[-2,1.5], [0,3]]))